# ChemML Wrapper Input File Generator

This is a graphical user interface (GUI) for the ChemML Wrapper.

* run gui in the **notebook** with two lines of code:
<pre>
```python
from cheml import wrapperGUI
ui = wrapperGUI()
```

</pre>

* You will see a graphical visualization of the workflow simultaneously. 
    - The numbers in the nodes represent the block id. 



In [1]:
import copy
import ipywidgets as widgets
from IPython.display import display, clear_output
from cheml.wrappers.database import sklearn_db, cheml_db
from cheml.wrappers.database.TSHF import tshf
from graphviz import Digraph


In [9]:

##########################################################

class container_page(object):
    def __init__(self, title, index, widget, block_params={}):
        self.title = title
        self.index = index
        self.widget = widget
        self.block_params = block_params

class ui(object):
    def __init__(self):
        self.tasks, self.combinations = tshf()  # task, subtask, host, function connections; keep the tasks for the order
        self.blocks = {}  # similar to cmls in the parser, {'task':{}, 'subtask':{}, 'host':{}, 'function':{}, 'parameters':{}, 'send':{'NA':'here'}, 'recv':{'NA':'here'} }
        self.block_id = 1
        self.comp_graph = []    # list of (iblock_send,token,iblock_recv,token)
        self.out_dir = "CMLWrapper.out"
        self.pages = {}
        self.graph = widgets.Image()
        self.accordion = widgets.Accordion()
        self.home_page()

    ################################

    def display_accordion(self, active_id=0, layout=widgets.Layout(border='solid gray 2px')):
        children = [self.pages[i].widget for i in sorted(self.pages)]
        self.accordion.children = children
        self.accordion.selected_index = active_id
        self.accordion.layout = layout
        new_titles = {}
        for i,ib in enumerate(sorted(self.pages)):
            if i in [0, 1]:
                self.accordion.set_title(i, '%s' % self.pages[i].title)
                # new_titles[str(i)] = '%s' % self.pages[i].title
            else:
                self.accordion.set_title(i, 'block# %i: %s' % (ib, self.pages[ib].title))
                # new_titles[str(i)] = 'block# %i: %s' % (ib, self.pages[ib].title)
        # self.accordion._titles = new_titles

    def close_accordion(self):
        pass
        # self.accordion.close()

    ################################

    def templates_widgets(self):
        headerT = widgets.Label(value='Start with a template workflow', layout=widgets.Layout(width='50%'))
        outdir = widgets.Text(
            value='CMLWrapper.out',
            placeholder='Type something',
            description='Output directory:',
            disabled=False,
        layout = widgets.Layout(margin='30px 0px 30px 0px')
        )
        def on_selectT1_clicked(b):
            self.output_directory = outdir.value
            self.add_page()
        def on_viewT1_clicked(b):
            self.output_directory = outdir.value
            self.add_page()

        t1 = widgets.Label(value="Template 1: check this paper for more information", layout=widgets.Layout(width='70%'))
        selectT1 = widgets.Button(description="Select")
        selectT1.style.button_color = 'lightblue'
        selectT1.on_click(on_selectT1_clicked)
        viewT1 = widgets.Button(description="Overview")
        viewT1.style.button_color = 'lightblue'
        viewT1.on_click(on_viewT1_clicked)
        hboxT1 = widgets.HBox([t1, selectT1, viewT1])

        def on_selectT2_clicked(b):
            self.output_directory = outdir.value
            self.add_page()
        def on_viewT2_clicked(b):
            self.output_directory = outdir.value
            self.add_page()

        t2 = widgets.Label(value="Template 2: check the overview for more information", layout=widgets.Layout(width='70%'))
        selectT2 = widgets.Button(description="Select")
        selectT2.style.button_color = 'lightblue'
        selectT2.on_click(on_selectT2_clicked)
        viewT2 = widgets.Button(description="Overview")
        viewT2.style.button_color = 'lightblue'
        viewT2.on_click(on_viewT2_clicked)
        hboxT2 = widgets.HBox([t2, selectT2, viewT2])

        vb = widgets.VBox([headerT, outdir, hboxT1, hboxT2])
        return vb

    def home_page_widgets(self):
        def on_selectN_clicked(b):
            self.output_directory = outdir.value
            self.add_page()

        def on_selectE_clicked(b):
            print txtarea.value
            # self.parser(txtarea.value)
            self.add_page()

        header = widgets.Label(value='Choose how to start ...', layout=widgets.Layout(width='50%'))
        # Tab: new script
        outdir = widgets.Text(
            value='CMLWrapper.out',
            placeholder='Type something',
            description='Output directory:',
            disabled=False,
            layout = widgets.Layout(margin='30px 0px 10px 0px'))
        headerN = widgets.Label(value='Start with a new script', layout=widgets.Layout(width='50%'))
        selectN = widgets.Button(description="Start", layout=widgets.Layout(margin='20px 0px 10px 115px'))
        selectN.style.button_color = 'lightblue'
        selectN.on_click(on_selectN_clicked)
        vboxN = widgets.VBox([headerN,outdir,selectN])
        # Tab: existing script
        headerE = widgets.Label(value='Load an existing script', layout=widgets.Layout(width='50%'))
        txtarea = widgets.Textarea(
            placeholder='copy a ChemML script (config file) here',
            disabled=False,
            layout = widgets.Layout(width='50%'))
        selectE = widgets.Button(description="Load", layout=widgets.Layout(margin='20px 0px 10px 115px'))
        selectE.style.button_color = 'lightblue'
        selectE.on_click(on_selectE_clicked)
        vboxE = widgets.VBox([headerE, txtarea, selectE])
        # Tab: template
        vboxT = self.templates_widgets()

        tabs = widgets.Tab()
        tabs.children = [vboxN, vboxE, vboxT]
        tabs.set_title(0, 'New script')
        tabs.set_title(1, 'Existing script')
        tabs.set_title(2, 'Template workflow')

        self.home_page_VBox = widgets.VBox([header, tabs])

    def home_page(self):
        id = 0
        # clear_output()
        self.home_page_widgets()
        self.pages[id] = container_page('Home page', id, self.home_page_VBox)
        self.display_accordion(id)
        display(self.accordion)
        print 'The computation graph will be displayed here:'
        display(self.graph)

    ################################

    def add_page_widgets(self):
        def on_select_clicked(b):
            self.block_id += 1
            block_params = {'task': task_w.value, 'subtask': subtask_w.value, \
                                          'host': host_w.value, 'function': func_w.value, \
                                          'wparams':{}, 'fparams':{}, 'inputs':{}, 'outputs':{}}
            self.pages[self.block_id] = container_page('%s'%func_w.value, self.block_id, None, block_params)
            self.current_bid = self.block_id
            self.debut = True
            self.custom_function_page()

        def _subtask_update():
            subtask_opts = [i for i in self.combinations[task_w.value]]
            subtask_w.options = subtask_opts
            subtask_w.value = subtask_opts[0]

        def _host_update():
            host_opts = [i for i in self.combinations[task_w.value][subtask_w.value]]
            host_w.options = host_opts
            host_w.value = host_opts[0]

        def _func_update():
            func_opts = [i for i in self.combinations[task_w.value][subtask_w.value][host_w.value]]
            func_w.options = func_opts
            func_w.value = func_opts[0]

        def handle_task_change(t):
            _subtask_update()
            _host_update()
            _func_update()

        def handle_subtask_change(s):
            _host_update()
            _func_update()

        def handle_host_change(h):
            _func_update()

        header = widgets.Label(value='Choose a method:', layout=widgets.Layout(width='50%'))
        task_options = self.tasks[1:3]
        task_w = widgets.Dropdown(
            options=task_options,
            value=task_options[0],
            description='Task:')
        subtask_options = [i for i in self.combinations[task_w.value]]
        subtask_w = widgets.Dropdown(
            options=subtask_options,
            value=subtask_options[0],
            description='Subtask:')
        host_options = [i for i in self.combinations[task_w.value][subtask_w.value]]
        host_w = widgets.Dropdown(
            options=host_options,
            value=host_options[0],
            description='Host:')
        func_options = [i for i in self.combinations[task_w.value][subtask_w.value][host_w.value]]
        func_w = widgets.Dropdown(
            options=func_options,
            value=func_options[0],
            description='Function:')
        select = widgets.Button(description="Select", layout=widgets.Layout(margin='20px 0px 10px 115px'))
        select.style.button_color = 'lightblue'
        select.on_click(on_select_clicked)

        task_w.observe(handle_task_change, names='value')
        subtask_w.observe(handle_subtask_change, names='value')
        host_w.observe(handle_host_change, names='value')

        self.add_page_VBox = widgets.VBox([header, task_w, subtask_w, host_w, func_w, select])

    def add_page(self):
        id = 1
        self.add_page_widgets()
        self.pages[id] = container_page('Add a block', id, self.add_page_VBox)
        self.display_accordion(id)

    ################################

    def db_extract_function(self, host, function):
        if host == 'sklearn':
            metadata = getattr(sklearn_db, function)()
        elif host == 'cheml':
            metadata = getattr(cheml_db, function)()
        wparams = {i:copy.deepcopy(vars(metadata.WParameters)[i]) for i in vars(metadata.WParameters).keys() if
                     i not in ('__module__', '__doc__')}
        fparams = {i:copy.deepcopy(vars(metadata.FParameters)[i]) for i in vars(metadata.FParameters).keys() if
                     i not in ('__module__', '__doc__')}
        inputs = {i: copy.deepcopy(vars(metadata.Inputs)[i]) for i in vars(metadata.Inputs).keys() if
                  i not in ('__module__', '__doc__')}
        outputs = {i: copy.deepcopy(vars(metadata.Outputs)[i]) for i in vars(metadata.Outputs).keys() if
                   i not in ('__module__', '__doc__')}
        return wparams, fparams, inputs, outputs, metadata

    def custom_function_IO_w(self):
        if not self.debut:
            self.current_bid = sorted(self.pages)[self.accordion.selected_index]

        def remove_nodes_toSEND(n = self.current_bid, remove_bids = set()):
            # find all the nodes that the current node can not send info to them
            remove_bids.add(n)  # including itself
            for e in self.comp_graph:
                if e[2]==n:
                    remove_bids.add(e[0])
                    remove_nodes_toSEND(n=e[0], remove_bids=remove_bids)
            return remove_bids

        def remove_nodes_toRECV(n = self.current_bid, remove_bids = set()):
            # find all the nodes that the current node can not receive info from them
            remove_bids.add(n)  # including itself
            for e in self.comp_graph:
                if e[0]==n:
                    remove_bids.add(e[2])
                    remove_nodes_toRECV(n=e[2], remove_bids=remove_bids)
            return remove_bids

        def refresh_tasks():
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            ## block ids that can send info to the current block
            bidS_options = [i for i in self.pages if i not in [0, 1]]
            rm = remove_nodes_toSEND(self.current_bid,set())
            for n in rm:
                bidS_options.remove(n)
            bidS.options = bidS_options

            ## block ids that can receive info from the current block
            bidR_options = [i for i in self.pages if i not in [0, 1]]
            rm = remove_nodes_toRECV(self.current_bid,set())
            for n in rm:
                bidR_options.remove(n)
            bidR.options = bidR_options

            ## update pipes options from comp_graph
            ps = ["%i, %s      >>>      %i, %s" % (e[0], e[1], e[2], e[3]) for e in self.comp_graph if self.current_bid in e]
            pipes.options = ps

            ## update input tokens to receive only once
            input_tokens = [token for token in self.pages[self.current_bid].block_params['inputs']]
            rm = []
            for t in input_tokens:
                for e in self.comp_graph:
                    if e[2:] == (self.current_bid, t):
                        rm.append(t)
            for token in rm:
                input_tokens.remove(token)
            receiver.options = input_tokens

            ## update external input tokens to receive only once
            if bidS.value is not None:
                external_inputs = [token for token in self.pages[bidS.value].block_params['inputs']]
                rm = []
                for t in external_inputs:
                    for e in self.comp_graph:
                        if e[2:] == (bidS.value, t):
                            rm.append(t)
                for token in rm:
                    external_inputs.remove(token)
                external_receivers.options = external_inputs
            else:
                external_receivers.options = []

            ## refresh external output tokens based on bidR value
            if bidR.value is not None:
                external_outputs = [token for token in self.pages[bidR.value].block_params['outputs']]
                external_senders.options = external_outputs
            else:
                external_senders.options = []

            ## clear ouput and update the graph viz
            # self.graph.close()
            # dot = Digraph(format='png')
            # for edge in self.comp_graph:
            #     dot.node('%i' % edge[0], label='%i %s' % (edge[0], self.pages[edge[0]].title))
            #     dot.node('%i' % edge[2], label='%i %s' % (edge[2],self.pages[edge[2]].title))
            #     dot.edge('%i' % edge[0], '%i'%edge[2], label='%s > %s' % (edge[1], edge[3]), labelfontcolor='green')
            # self.graph = widgets.Image(value=dot.pipe(),format='png')
            # display(self.graph)

        def on_refresh_clicked(b):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            refresh_tasks()

        def on_addS_clicked(b):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            if sender.value is not None and bidS.value is not None and external_receivers.value is not None:
                input_type = set(self.pages[bidS.value].block_params['inputs'][external_receivers.value].types)
                output_type = set(self.pages[self.current_bid].block_params['outputs'][sender.value].types)
                if input_type.issubset(output_type) or output_type.issubset(input_type):
                    all_receivers = [e[2:] for e in self.comp_graph]
                    if (bidS.value, external_receivers.value) not in all_receivers:
                        edge = (self.current_bid, sender.value, bidS.value, external_receivers.value)
                        if edge not in self.comp_graph:
                            self.comp_graph.append(edge)
            ## all refresh tasks:
            refresh_tasks()

        def on_addR_clicked(b):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            if receiver.value is not None and bidR.value is not None and external_senders.value is not None:
                output_type = set(self.pages[bidR.value].block_params['outputs'][external_senders.value].types)
                input_type = set(self.pages[self.current_bid].block_params['inputs'][receiver.value].types)
                if input_type.issubset(output_type) or output_type.issubset(input_type):
                    all_receivers = [e[2:] for e in self.comp_graph]
                    if (self.current_bid, receiver.value) not in all_receivers:
                        edge = (bidR.value, external_senders.value, self.current_bid, receiver.value)
                        if edge not in self.comp_graph:
                            self.comp_graph.append(edge)
            ## all refresh tasks:
            refresh_tasks()

        def bidS_value_change(change):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            if bidS.value is not None:
                external_inputs = [token for token in self.pages[bidS.value].block_params['inputs']]
                rm = []
                for t in external_inputs:
                    for e in self.comp_graph:
                        if e[2:] == (bidS.value, t):
                            rm.append(t)
                for token in rm:
                    external_inputs.remove(token)
                external_receivers.options = external_inputs
            else:
                external_receivers.options = []

        def bidR_value_change(change):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            if bidR.value is not None:
                external_outputs = [token for token in self.pages[bidR.value].block_params['outputs']]
                external_senders.options = external_outputs
            else:
                external_senders.options = []

        def on_remove_clicked(b):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            rm = pipes.value
            for p in rm:
                p = p.strip().split(',')
                m = p[1].strip().split('      >>>      ')
                edge = (int(p[0]), m[0].strip(), int(m[1]), p[2].strip())
                if edge in self.comp_graph:
                    self.comp_graph.remove(edge)
            refresh_tasks()

        ## Sender
        output_tokens = [token for token in self.pages[self.current_bid].block_params['outputs']]
        headerS = widgets.HTML(value='<b> Send >>> </b>', layout=widgets.Layout(width='50%',margin='10px 0px 0px 10px'))
        sender = widgets.Dropdown(
            options=output_tokens,
            # value=output_tokens[0],
            description='output token:')
        hbox1S = widgets.HBox([sender],layout=widgets.Layout(height='40px', border='dotted black 1px',
                                                               align_items='center',  # justify_content = 'center',
                                                               margin='0px 0px 0px 10px'))
        toS = widgets.HTML(value='<b> >>> </b>')
        bidS_options = [i for i in self.pages if i not in [0, 1]]
        rm = remove_nodes_toSEND(self.current_bid,set())
        for n in rm:
            bidS_options.remove(n)
        bidS = widgets.Dropdown(
            options=bidS_options,
            description='block#:',
            layout=widgets.Layout(width='140px'))
        bidS.observe(bidS_value_change,names='value')
        refreshS = widgets.Button(description="Refresh", layout=widgets.Layout(width='60px'))
        refreshS.style.button_color = 'darkseagreen'
        refreshS.on_click(on_refresh_clicked)
        if bidS.value is not None:
            external_inputs = [token for token in self.pages[bidS.value].block_params['inputs']]
            rm = []
            for t in external_inputs:
                for e in self.comp_graph:
                    if e[2:] == (bidS.value, t):
                        rm.append(t)
            for token in rm:
                external_inputs.remove(token)
        else:
            external_inputs = []
        external_receivers = widgets.Dropdown(
            options = external_inputs,
            description='input token:')
        hbox2S = widgets.HBox([bidS,refreshS, external_receivers],
                             layout=widgets.Layout(height='40px', border='dotted black 1px',
                                                   align_items='center', justify_content='center'))
        addS = widgets.Button(description="Add", layout=widgets.Layout(width='60px', margin='0px 10px 0px 0px'))
        addS.style.button_color = 'lightblue'
        addS.on_click(on_addS_clicked)
        hboxS = widgets.HBox([hbox1S, toS, hbox2S, addS],
                             layout=widgets.Layout(justify_content='space-between',
                                                   align_items='center',
                                                   margin='10px 0px 20px 0px'))

        ## Receiver
        note = widgets.HTML(value='<b> Note: </b> This page automatically avoid: (1) loops, (2) type inconsistency, and (3) more than one input per token.', layout=widgets.Layout(margin='20px 0px 0px 10px'))
        headerR = widgets.HTML(value='<b> Receive <<< </b>', layout=widgets.Layout(width='50%',margin='10px 0px 0px 10px'))
        input_tokens = [token for token in self.pages[self.current_bid].block_params['inputs']]
        rm = []
        for t in input_tokens:
            for e in self.comp_graph:
                if e[2:] == (self.current_bid, t):
                    rm.append(t)
        for token in rm:
            input_tokens.remove(token)
        receiver = widgets.Dropdown(
            options=input_tokens,
            # value=output_tokens[0],
            description='input token:')
        hbox1R = widgets.HBox([receiver],layout=widgets.Layout(height='40px', border='dotted black 1px',
                                                               align_items='center',  # justify_content = 'center',
                                                               margin='0px 0px 0px 10px'))
        fro = widgets.HTML(value='<b> <<< </b>')
        bidR_options = [i for i in self.pages if i not in [0, 1]]
        rm = remove_nodes_toRECV(self.current_bid,set())
        for n in rm:
            bidR_options.remove(n)
        bidR = widgets.Dropdown(
            options=bidR_options,
            description='block#:',
            layout=widgets.Layout(width='140px'))
        bidR.observe(bidR_value_change,names='value')
        refresh = widgets.Button(description="Refresh", layout=widgets.Layout(width='60px'))
        refresh.style.button_color = 'darkseagreen'
        refresh.on_click(on_refresh_clicked)
        if bidR.value is not None:
            external_outputs = [token for token in self.pages[bidR.value].block_params['outputs']]
        else:
            external_outputs = []
        external_senders = widgets.Dropdown(
            options = external_outputs,
            description='output token:')
        hbox2R = widgets.HBox([bidR, refresh,external_senders],
                             layout=widgets.Layout(height='40px', border='dotted black 1px',
                                                   align_items='center', justify_content='center'))
        addR = widgets.Button(description="Add", layout=widgets.Layout(width='60px', margin='0px 10px 0px 0px'))
        addR.style.button_color = 'lightblue'
        addR.on_click(on_addR_clicked)
        hboxR = widgets.HBox([hbox1R, fro, hbox2R, addR],
                             layout=widgets.Layout(justify_content='space-between',
                                                   align_items='center',
                                                   margin='10px 0px 20px 0px'))

        ## vbox
        pipes_options = [edge for edge in self.comp_graph if self.current_bid in edge]
        pipes = widgets.SelectMultiple(
            options=pipes_options,
            # description='list of pipes:',
            layout=widgets.Layout(margin='20px 0px 20px 250px'))
        remove = widgets.Button(description="Remove pipe",
                                layout=widgets.Layout(width='120px', margin='40px 0px 0px 10px'))
        remove.style.button_color = 'lightblue'
        remove.on_click(on_remove_clicked)
        hbox4 = widgets.HBox([pipes, remove], margin='0px 0px 0px 100px')

        IO_vbox = widgets.VBox([note,headerR, hboxR, headerS, hboxS, hbox4])#, layout=widgets.Layout(border='solid darkslategray 1px'))
        return IO_vbox

    def custom_function_params_w(self):
        # def handle_param_change(b):
        #     for item in self.pages[current_bid].block_params['wparams']:
        #         self.pages[current_bid].block_params['wparams'][item].value = self.pages[current_bid].block_params['wparams'][item].widget.value
        #     for item in self.pages[current_bid].block_params['fparams']:
        #         self.pages[current_bid].block_params['fparams'][item].value = self.pages[current_bid].block_params['fparams'][item].widget.value
        if not self.debut:
            self.current_bid = sorted(self.pages)[self.accordion.selected_index]

        def on_default_clicked(b):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            for item in self.pages[self.current_bid].block_params['wparams']:
                self.pages[self.current_bid].block_params['wparams'][item].widget.value = \
                str(self.pages[self.current_bid].block_params['wparams'][item].default)
            for item in self.pages[self.current_bid].block_params['fparams']:
                self.pages[self.current_bid].block_params['fparams'][item].widget.value = \
                str(self.pages[self.current_bid].block_params['fparams'][item].default)

        # wrapper parameters widgets
        header = widgets.HTML(value='<b>Wrapper parameters:</b>', layout=widgets.Layout(width='50%'))
        wparams = self.pages[self.current_bid].block_params['wparams']
        wparams_boxes = []
        for item in wparams:
            wp = widgets.Text(
                value=str(wparams[item].default),
                placeholder=str(wparams[item].options),
                description=wparams[item].name,
                disabled=False,
                layout = widgets.Layout(width='40%'))
            wparams[item].widget = wp
            # wp.observe(handle_param_change, names='value')
            wformat = widgets.Text(
                value=str(wparams[item].format),
                description= 'Type:',
                disabled=True)
            hbox = widgets.HBox([wp,wformat])
            wparams_boxes.append(hbox)
        wparams_vbox = widgets.VBox([header]+ wparams_boxes)
        self.pages[self.current_bid].block_params['wparams'] = wparams

        # function parameters widgets
        header = widgets.HTML(value='<b>Function parameters:</b>',
                              layout=widgets.Layout(width='50%',margin='20px 0px 0px 0px'))
        fparams = self.pages[self.current_bid].block_params['fparams']
        fparams_boxes = []
        for item in fparams:
            wp = widgets.Text(
                value=str(fparams[item].default),
                placeholder=str(fparams[item].options),
                description=fparams[item].name,
                disabled=False,
                layout=widgets.Layout(width='40%'))
            fparams[item].widget = wp
            # wp.observe(handle_param_change, names='value')
            wformat = widgets.Text(
                value=str(fparams[item].format),
                description='Type:',
                disabled=True)
            hbox = widgets.HBox([wp, wformat])
            fparams_boxes.append(hbox)
        fparams_vbox = widgets.VBox([header]+ fparams_boxes)
        self.pages[self.current_bid].block_params['fparams'] = fparams

        defaultB = widgets.Button(description="Default values",
                                  layout=widgets.Layout(width='110px',margin='10px 0px 10px 180px'))
        defaultB.style.button_color = 'lightblue'
        defaultB.on_click(on_default_clicked)

        params_vbox = widgets.VBox([wparams_vbox,fparams_vbox,defaultB],
                                   layout=widgets.Layout(justify_content='center', align_content='center'))

        return params_vbox

    def set_default_params_IO(self):
        if not self.debut:
            self.current_bid = sorted(self.pages)[self.accordion.selected_index]
        host = self.pages[self.current_bid].block_params['host']
        function = self.pages[self.current_bid].block_params['function']
        wparams, fparams, inputs, outputs, metadata = self.db_extract_function(host,function)
        self.pages[self.current_bid].block_params['wparams'] = wparams
        self.pages[self.current_bid].block_params['fparams'] = fparams
        self.pages[self.current_bid].block_params['inputs'] = inputs
        self.pages[self.current_bid].block_params['outputs'] = outputs
        self.pages[self.current_bid].block_params['requirements'] = metadata.requirements
        self.pages[self.current_bid].block_params['documentation'] = metadata.documentation

    def custom_function_widgets(self):
        if not self.debut:
            self.current_bid = sorted(self.pages)[self.accordion.selected_index]

        def on_remove_clicked(b):
            if not self.debut:
                self.current_bid = sorted(self.pages)[self.accordion.selected_index]
            del self.pages[self.current_bid]
            rm = [edge for edge in self.comp_graph if self.current_bid in edge]
            for e in rm:
                self.comp_graph.remove(e)

            ## clear ouput and update the graph viz
            self.graph.close()
            dot = Digraph(format='png')
            for edge in self.comp_graph:
                dot.node('%i' % edge[0], label='%i %s' % (edge[0], self.pages[edge[0]].title))
                dot.node('%i' % edge[2], label='%i %s' % (edge[2],self.pages[edge[2]].title))
                dot.edge('%i' % edge[0], '%i'%edge[2], label='%s > %s' % (edge[1], edge[3]), labelfontcolor='green')
            self.graph = widgets.Image(value=dot.pipe(),format='png')
            display(self.graph)


            self.add_page()

        # initialize
        if len(self.pages[self.current_bid].block_params['fparams']) == len(self.pages[self.current_bid].block_params['inputs']) ==\
                len(self.pages[self.current_bid].block_params['outputs']) == 0:
            self.set_default_params_IO()
        # parameters widgets
        params_vbox = self.custom_function_params_w()

        # inputs and outputs
        IO_vbox = self.custom_function_IO_w()

        removeB = widgets.Button(description="Remove block",layout=widgets.Layout(margin='10px 0px 10px 400px'))
        removeB.style.button_color = 'lightblue'
        removeB.on_click(on_remove_clicked)

        custom_f_accordion = widgets.Tab(layout=widgets.Layout(margin='10px 0px 10px 0px'))
        custom_f_accordion.children = [params_vbox, IO_vbox]
        custom_f_accordion.set_title(0, 'Parameters')
        custom_f_accordion.set_title(1, 'Input/Output')
        # custom_f_accordion.selected_index = 0

        custom_f_vbox = widgets.VBox([custom_f_accordion, removeB],
                                     layout=widgets.Layout(justify_content='center', align_content='center'))

        return custom_f_vbox

    def custom_function_page(self):
        custom_function_VBox = self.custom_function_widgets()
        self.pages[self.current_bid].widget = custom_function_VBox
        # display
        active_id = [i for i in sorted(self.pages)].index(self.current_bid)
        active_id = 1
        self.display_accordion(active_id)
        self.debut = False

gui = ui()


# comp_graph = [[0, 'df', 2, 'df'], [0, 'api', 1, 'iv1'], [1, 'ov1', 3, 'df2'], [2, 'df', 3, 'df1'], [1, 'ov2', 4, 'df']]



Accordion(children=(VBox(children=(Label(value=u'Choose how to start ...', layout=Layout(width=u'50%')), Tab(children=(VBox(children=(Label(value=u'Start with a new script', layout=Layout(width=u'50%')), Text(value=u'CMLWrapper.out', description=u'Output directory:', layout=Layout(margin=u'30px 0px 10px 0px'), placeholder=u'Type something'), Button(description=u'Start', layout=Layout(margin=u'20px 0px 10px 115px'), style=ButtonStyle(button_color='lightblue')))), VBox(children=(Label(value=u'Load an existing script', layout=Layout(width=u'50%')), Textarea(value=u'', layout=Layout(width=u'50%'), placeholder=u'copy a ChemML script (config file) here'), Button(description=u'Load', layout=Layout(margin=u'20px 0px 10px 115px'), style=ButtonStyle(button_color='lightblue')))), VBox(children=(Label(value=u'Start with a template workflow', layout=Layout(width=u'50%')), Text(value=u'CMLWrapper.out', description=u'Output directory:', layout=Layout(margin=u'30px 0px 30px 0px'), placeholder=u'Type something'), HBox(children=(Label(value=u'Template 1: check this paper for more information', layout=Layout(width=u'70%')), Button(description=u'Select', style=ButtonStyle(button_color='lightblue')), Button(description=u'Overview', style=ButtonStyle(button_color='lightblue')))), HBox(children=(Label(value=u'Template 2: check the overview for more information', layout=Layout(width=u'70%')), Button(description=u'Select', style=ButtonStyle(button_color='lightblue')), Button(description=u'Overview', style=ButtonStyle(button_color='lightblue'))))))), _titles={u'1': 'Existing script', u'0': 'New script', u'2': 'Template workflow'}))),), layout=Layout(border=u'solid gray 2px'), _titles={u'0': 'Home page'})

The computation graph will be displayed here:


Image(value='')